In [2]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
import sys

In [3]:
t1 = time.time()
df_train = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_cm0410time_last75m.csv')

t2 = time.time()
print('training loading done! Time: {}'.format(t2 - t1))
print('size is: {}'.format(sys.getsizeof(df_train) / 1024 ** 3))
print('length is {}'.format(len(df_train)))

training loading done! Time: 422.89514923095703
size is: 32.4100256934762
length is 75000000


In [4]:
# ### 0410 add one more col
# train_ip = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/train_ip_col.csv')
# df_train['ip_extra_hz'] = train_ip['val']
# del train_ip

# Specify Categorical Columns

In [5]:
df_train.columns

Index(['ip_day_intesthh_count', 'ip_day_hour_count', 'ip_os_day_hour_count',
       'ip_app_day_hour_count', 'ip_app_os_day_hour_count',
       'app_day_hour_count', 'app_channel_count', 'device_os_count',
       'app_os_channel_hour_count', 'app_os_channel_mean',
       'app_os_channel_intesthh_mean', 'ip_mean', 'ip_app_mean',
       'ip_device_channel_mean', 'ip_app_device_mean', 'ip_app_intesthh_mean',
       'ip_channel_mean', 'ip_os_mean', 'ip_device_mean',
       'ip_app_device_channel_mean', 'ip_app_device_channel_time2nextclick',
       'ip_app_device_os_time2nextclick',
       'app_device_os_channel_time2nextclick',
       'ip_app_device_os_channel_time2nextclick', 'ip_time2nextclick',
       'ip_time2previousclick', 'ip_app_time2previousclick',
       'ip_device_time2previousclick', 'ip_channel_time2previousclick',
       'ip_app_device_time2previousclick', 'ip_os_time2previousclick',
       'ip_app_device_os_channel_time2previousclick',
       'app_device_countfromfuture', '

In [6]:
categorical_col = [ 'app', 'device', 'os', 'channel', 'hour', ]
target = 'is_attributed'
exclude_list = [
#                 
#                 'app_os_channel_intesthh_mean', 
                'app_device_os_channel_mean', #may not use
                'app_device_os_channel_intesthh_mean', # may not use
                'app_channel_intesthh_mean', # may not use
                'app_device_os_mean', # may not use
                'app_channel_mean', # may not use
    
                'ip_app_device_channel_mean',  # may not use
                'ip_app_device_os_mean', # may not sue
    
#                 'ip_os_mean',   #97.8529     
#                 'ip_device_mean', #97.8529
#                 'ip_mean', 
#                 'ip_app_mean',
#                 'ip_device_channel_mean', 
#                 'ip_app_device_mean' , 
#                 'ip_app_intesthh_mean', 
#                 'ip_channel_mean'   #0.9713
#                 'app_os_channel_mean',  
]

# ['ip_os_mean', 'ip_device_mean'] ~ 0.978439   ~lb0.9713
#
#########################################################
# exclude ['ip_app_device_channel_mean'] ~ 0.978669


###########################################################

# exclude ['app_os_channel_intesthh_mean']  0.978744

exclude_list = []


# features_mean = [
#                 'app_os_channel_mean',  
#                 'app_os_channel_intesthh_mean', 
#                 'app_device_os_channel_mean',
#                 'app_device_os_channel_intesthh_mean', 
#                 'app_channel_intesthh_mean',
#                 'ip_mean', 
#                 'ip_app_mean',
#                 'ip_app_device_channel_mean', 
#                 'ip_device_channel_mean', 
#                 'ip_app_device_mean' , 
#                 'ip_app_intesthh_mean', 
#                 'ip_channel_mean',

#     'ip_os_mean',  
#     'ip_app_device_os_mean',
#     'ip_device_mean',
#     'app_device_os_mean', 
#     'app_channel_mean']


remain = ['ip_mean']
remain = ['ip_mean', 
                'ip_app_mean',
                'ip_app_device_channel_mean', 
                'ip_device_channel_mean', 
                'ip_app_device_mean' , 
                'ip_app_intesthh_mean', 
                'ip_channel_mean']
feature_cols = list(set(df_train.columns) - set([target]) -set(exclude_list))

# extra = ['ip_extra_hz']
extra = []
feature_cols.extend(extra)
feature_cols

['ip_device_time2previousclick',
 'app_channel_count',
 'device_countfromfuture',
 'ip_device_mean',
 'ip_os_mean',
 'ip_app_day_hour_count',
 'ip_mean',
 'ip_channel_time2previousclick',
 'ip_app_device_channel_time2nextclick',
 'ip_time2previousclick',
 'ip_app_device_time2previousclick',
 'app_countfrompast',
 'ip_app_lasttimediff',
 'app_lasttimediff',
 'hour',
 'ip_app_device_os_channel_time2previousclick',
 'ip_app_device_channel_mean',
 'ip_day_intesthh_count',
 'ip_app_os_day_hour_count',
 'ip_app_device_os_channel_time2nextclick',
 'device_lasttimediff',
 'ip_app_device_os_countfrompast',
 'ip_day_hour_count',
 'app_day_hour_count',
 'ip_app_intesthh_mean',
 'app',
 'ip_os_lasttimediff',
 'os',
 'ip_app_device_os_channel_countfrompast',
 'ip_channel_mean',
 'app_os_channel_intesthh_mean',
 'ip_app_time2previousclick',
 'app_os_channel_hour_count',
 'channel',
 'ip_app_device_os_lasttimediff',
 'ip_device_channel_mean',
 'app_os_channel_mean',
 'ip_countfromfuture',
 'os_lastti

In [7]:
# # remove minute and second
# categorical_col = ['app', 'device', 'os', 'channel', 'hour']
# target = 'is_attributed'
# feature_cols = list(set(df_train.columns) - set([target]) - set(['minute', 'second', 'ip']))

In [8]:
# # remove minute and second
# categorical_col = [ 'device' ]
# target = 'is_attributed'
# feature_cols = list(set(df_train.columns) - set([target]) - set(['minute', 'second', 'ip', 'app', 'os', 'channel', 'hour']))

# Get Validation -- Last 10%

In [9]:
# ratio = 0.3
# length = len(df_train)
# front = int( (1-ratio) * length)
# trainset = df_train.iloc[:front]
# valset = df_train.iloc[front:]

# print(sys.getsizeof(trainset)/ 1024 **3)
# print(len(trainset))

# Get Validation -- Head 10%

In [10]:
ratio = 16/75
ratio = 0.3
length = len(df_train)
front = int( ratio * length)

valset = df_train.iloc[:front]
trainset = df_train.iloc[front:]


print(sys.getsizeof(trainset)/ 1024 **3)
print(len(trainset))

22.687018018215895
52500000


# Get Validation -- random 10 %

In [11]:
# from sklearn.model_selection import train_test_split

# ratio = 0.3

# trainset, valset = train_test_split(df_train,test_size=ratio, random_state=19)
# print(sys.getsizeof(trainset)/ 1024 **3)
# print(len(trainset))

In [12]:
y_train = trainset[target].values
y_val = valset[target].values

lgb_train = lgb.Dataset(trainset[feature_cols], y_train, categorical_feature = categorical_col)
lgb_val = lgb.Dataset(valset[feature_cols], y_val, categorical_feature = categorical_col)

zeros = len(y_train[y_train == 0])
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train) * 100
import gc
# del df_train
gc.collect()

print('train negative rate: {}'.format(scale_pos_weight))
print('val negative rate: {}'.format(len(y_val[y_val == 0]) / len(y_val) * 100))

train negative rate: 99.74658857142857
val negative rate: 99.80628444444444


# Train Lightgbm

In [13]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 7,
        'num_threads': 6, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':100,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.964822
[20]	valid_0's auc: 0.968632
[30]	valid_0's auc: 0.970764
[40]	valid_0's auc: 0.972627
[50]	valid_0's auc: 0.973959
[60]	valid_0's auc: 0.97448
[70]	valid_0's auc: 0.974899
[80]	valid_0's auc: 0.975194
[90]	valid_0's auc: 0.975518
[100]	valid_0's auc: 0.975826
[110]	valid_0's auc: 0.975948
[120]	valid_0's auc: 0.975992
[130]	valid_0's auc: 0.976255
[140]	valid_0's auc: 0.976401
[150]	valid_0's auc: 0.976526
[160]	valid_0's auc: 0.976692
[170]	valid_0's auc: 0.976633
[180]	valid_0's auc: 0.976642
[190]	valid_0's auc: 0.976758
[200]	valid_0's auc: 0.97678
[210]	valid_0's auc: 0.976783
[220]	valid_0's auc: 0.9768
[230]	valid_0's auc: 0.976853
[240]	valid_0's auc: 0.976902
[250]	valid_0's auc: 0.976946
[260]	valid_0's auc: 0.976962
[270]	valid_0's auc: 0.97699
[280]	valid_0's auc: 0.977017
[290]	valid_0's auc: 0.97704
[300]	valid_0's auc: 0.977064
[310]	valid_0's auc: 0.97705
[320]	valid_0's auc: 0.

In [14]:
from sklearn.metrics import roc_auc_score
pred_val = model.predict(valset[feature_cols])
print(roc_auc_score(y_val,pred_val))

0.97713091296


In [15]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
importance.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/output/importance_sub_cm0410_4_leaf7weight.csv')

In [16]:
importance

channel                                        660
os                                             350
app                                            247
ip_app_os_day_hour_count                       136
hour                                            99
app_os_channel_mean                             99
ip_app_device_os_lasttimediff                   91
ip_countfrompast                                72
ip_app_device_os_channel_lasttimediff           69
ip_day_hour_count                               53
app_os_channel_intesthh_mean                    43
ip_day_intesthh_count                           42
app_countfrompast                               41
ip_device_mean                                  40
ip_device_countfrompast                         37
ip_app_device_os_countfrompast                  36
ip_os_lasttimediff                              35
ip_mean                                         33
ip_app_device_mean                              32
device                         

In [17]:
df_test = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_cm0410time_last75m.csv')

### 0410 add one more col
test_ip = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test_ip_col.csv')
df_test['ip_extra_hz'] = test_ip['val']
del test_ip

# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_test = df_test[list(valset[feature_cols].columns)]
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = model.predict(df_test)
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/test_cm0409_4_leaf7weighted.csv.gz', compression='gzip', index=False)


loading file done!
predicting file done!


In [22]:
a = ['ip_day_hour_count', 'ip', 'day', 'hour']
df_train.loc[:20,a ]

,ip_day_hour_count,ip,day,hour
0,43,143414,NaN,12
1,18,173096,NaN,12
2,42,8210,NaN,12
3,19,5746,NaN,12
4,65,31475,NaN,12
5,11,251465,NaN,12
6,164,163593,NaN,12
7,158,58288,NaN,12
8,134,27038,NaN,12
9,414,67682,NaN,12
